In [1]:
import os
# set the current working directory to the deployed package folder. This is required by isaac.
# This cell should only run once.
os.chdir("/home/davis/deploy/davis/simple_joint_control-pkg")
os.getcwd()

'/home/davis/deploy/davis/simple_joint_control-pkg'

In [2]:
from IPython.display import display
import json
import numpy as np
import time
import threading

from engine.pyalice import Application, Codelet
from engine.pyalice.gui.composite_widget import CompositeWidget
from engine.pyalice.Composite import parse_composite_message

np.set_printoptions(precision=3)

In [39]:
# A Python codelet for joint control through widget
class JointPositionControl(Codelet):
    def start(self):
        self.rx = self.isaac_proto_rx("CompositeProto", "state")
        self.tx = self.isaac_proto_tx("CompositeProto", "command")

        joints = self.config.joints
        limits = self.config.limits
        self._widget = CompositeWidget(joints, "position", limits)
        if self._widget is None:
            report_failure("Cannot create valid widget")
            return
        display(self._widget.panel)

        self.tick_periodically(0.1)

    def tick(self):
        state_msg = self.rx.message
        if state_msg is None:
            return
        
        print(parse_composite_message(state_msg, self._widget.composite._quantities))

        self._widget.composite = state_msg

        self.tx._msg = self._widget.composite
        if self.tx._msg is not None:
            self.tx.publish()

UR10 with Smarthand in Omniverse Isaac Sim
======


In [40]:
# set kinematic file and get list of joints
kinematic_file = "/home/davis/deploy/davis/rm_isaac_bridge-pkg/apps/assets/kinematic_trees/rm_ur10.kinematic.json"
joints = []
with open(kinematic_file,'r') as fd:
    kt = json.load(fd)
    for link in kt['links']:
        if 'motor' in link and link['motor']['type'] != 'constant':
            joints.append(link['name'])
print(joints)

['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint', 'left_finger', 'left_finger_joint', 'left_finger_upper', 'right_finger', 'right_finger_joint', 'right_finger_upper']


In [41]:
app = Application(name="simple_joint_control_smarthand_sim")

# load subgraphcs
app.load(filename="packages/planner/apps/multi_joint_lqr_control.subgraph.json", prefix="lqr")
app.load(filename="packages/navsim/apps/navsim_tcp.subgraph.json", prefix="simulation")
# edges
simulation_node = app.nodes["simulation.interface"]
lqr_interface = app.nodes["lqr.subgraph"]["interface"]
app.connect(simulation_node["output"], "joint_state", lqr_interface, "joint_state")
app.connect(lqr_interface, "joint_command", simulation_node["input"], "joint_position")
# configs
app.nodes["lqr.kinematic_tree"]["KinematicTree"].config.kinematic_file = kinematic_file
lqr_planner = app.nodes["lqr.local_plan"]["MultiJointLqrPlanner"]
lqr_planner.config.speed_min = [-50.0] * len(joints)
lqr_planner.config.speed_max = [50.0] * len(joints)
lqr_planner.config.acceleration_min = [-50.0] * len(joints)
lqr_planner.config.acceleration_max = [50.0] * len(joints)

# add pycodelet JointPositionControl
widget_node = app.add("command_generator")
joint_commander = widget_node.add(JointPositionControl)
joint_commander.config.joints = joints
joint_commander.config.limits = [[-7, 7]] * len(joints)
app.connect(joint_commander, "command", lqr_interface, "joint_target")
app.connect(simulation_node["output"], "joint_state", joint_commander, "state")

2020-11-13 11:36:02,799 DEBUG Binding PyCodelet command_generator/PyCodelet


In [42]:
app.start()

2020-11-13 11:36:03,279 DEBUG Launching isaac core
2020-11-13 11:36:03,280 DEBUG Launching pycodelet threads
2020-11-13 11:36:03,288 DEBUG Launching command_generator/PyCodelet


2020-11-13 11:36:03,597 DEBUG Stopped command_generator/PyCodelet


In [38]:
# stop Isaac app
app.stop()

2020-11-13 11:35:59,142 DEBUG Python Codelets All stopped...
2020-11-13 11:35:59,142 CRITICAL (<class 'ValueError'>, ValueError("Missing ['left_finger', 'position', 1] in composite message",), <traceback object at 0x7fc455e1e288>)


RuntimeError: Exceptions happened on PyCodelet Thread(s)

Kinova Jaco (gen2, 7 joints) Hardware
======

Install the KinoveJaco SDK in /opt/JACO2SDK (tested with v1.4.2) and connect to workstation via USB. Make sure the USB port has write permission

In [ ]:
kinematic_file = "apps/assets/kinematic_trees/kinova_j2n7.kinematic.json"
joints = []
with open(kinematic_file,'r') as fd:
    kt = json.load(fd)
    for link in kt['links']:
        if 'motor' in link and link['motor']['type'] != 'constant':
            joints.append(link['name'])
print(joints)

In [ ]:
app = Application(name="simple_joint_control_kinova_real")

# load lqr subgraphcs
app.load(filename="packages/planner/apps/multi_joint_lqr_control.subgraph.json", prefix="lqr")
lqr_interface = app.nodes["lqr.subgraph"]["interface"]
# add kinova driver codelet
app.load_module("kinova_jaco")
driver = app.add("driver").add(app.registry.isaac.kinova_jaco.KinovaJaco)
# edges
app.connect(driver, "arm_state", lqr_interface, "joint_state")
app.connect(lqr_interface, "joint_command", driver, "arm_command")
# configs
app.nodes["lqr.kinematic_tree"]["KinematicTree"].config.kinematic_file = kinematic_file
lqr_planner = app.nodes["lqr.local_plan"]["MultiJointLqrPlanner"]
lqr_planner.config.speed_min = [-0.5] * len(joints)
lqr_planner.config.speed_max = [0.5] * len(joints)
lqr_planner.config.acceleration_min = [-0.5] * len(joints)
lqr_planner.config.acceleration_max = [0.5] * len(joints)
driver.config.kinematic_tree = "lqr.kinematic_tree"
driver.config.kinova_jaco_sdk_path = "/opt/JACO2SDK/API/"
driver.config.tick_period = "50ms"

# add pycodelet JointPositionControl
widget_node = app.add("command_generator")
joint_commander = widget_node.add(JointPositionControl)
joint_commander.config.joints = joints
joint_commander.config.limits = [[-2*np.pi, 2*np.pi]] * len(joints)
app.connect(joint_commander, "command", lqr_interface, "joint_target")
app.connect(driver, "arm_state", joint_commander, "state")
                            
app.start()

In [ ]:
app.stop()